In [28]:
import torch
import bitsandbytes as bnb
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import Dataset

DATA_PATH = Path("../data")
OMI_PATH_processed = DATA_PATH / "processed" / "omi-health"
OMI_PATH_raw = DATA_PATH / "raw" / "omi-health"
MODEL_PATH =  Path("../models")

print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device name: {torch.cuda.get_device_name(0)}")

print(f"Bitsandbytes version: {bnb.__version__}")

PyTorch CUDA available: True
CUDA device count: 1
Current CUDA device name: NVIDIA RTX A4000
Bitsandbytes version: 0.45.5


In [3]:
model_id = "google/txgemma-2b-predict"

# Use 4-bit quantization to reduce memory usage
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map={"":0},
    torch_dtype="auto",
    attn_implementation="eager",
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\khal6952\.cache\huggingface\hub\models--google--txgemma-2b-predict. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Loading checkpoint shards: 

In [4]:
import pandas as pd

# Load the dataset
train_df = pd.read_csv(OMI_PATH_processed / "train_v1.csv")
train_df.head()

,dialogue,soap,prompt,messages,messages_nosystem,event_tags
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': ""You are an exper...",['(After the tests)']
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': ""You are an exper...","['[After the tests]', '[After 3 weeks of thera..."
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': ""You are an exper...",[]
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': ""You are an exper...",[]
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': ""You are an exper...",[]


In [22]:
def format_dialogue_for_soap_synthesis_v2(data):
    dialogue = data["dialogue"]
    soap = data["completion"]
    return f"dialogue: {dialogue}<soap_start> {soap} <eos>"

In [23]:
dataset = Dataset.from_pandas(train_df)
dataset = dataset.rename_column('soap', 'completion')

dataset

Dataset({
    features: ['dialogue', 'completion', 'prompt', 'messages', 'messages_nosystem', 'event_tags'],
    num_rows: 9250
})

In [24]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "o_proj",
        "k_proj",
        "v_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

In [25]:
from peft import prepare_model_for_kbit_training, get_peft_model

# Preprocess quantized model for training
model = prepare_model_for_kbit_training(model)

# Create PeftModel from quantized model and configuration
model = get_peft_model(model, lora_config)

C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [26]:
import transformers
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=20,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        max_seq_length=512,
        output_dir="/content/outputs",
        optim="paged_adamw_8bit",
        report_to="none",
    ),
    peft_config=lora_config,
    formatting_func=format_dialogue_for_soap_synthesis_v2,
)

Truncating train dataset: 100%|██████████| 9250/9250 [00:00<00:00, 117028.57 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [27]:
trainer.train()

C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
5,16.453000
10,15.947400
15,13.262100
20,7.866200
25,5.936500
30,4.485400
35,3.885000
40,3.354300
45,3.045200
50,3.047300


TrainOutput(global_step=500, training_loss=2.756281015396118, metrics={'train_runtime': 1308.8063, 'train_samples_per_second': 1.528, 'train_steps_per_second': 0.382, 'total_flos': 1.1355701297624064e+16, 'train_loss': 2.756281015396118})

In [20]:
def generate_soap_note(dialogue, model, tokenizer, device="cuda:0"):
    """Generates a SOAP note from a given dialogue."""

    # Format the input (you might need to adapt this based on your training format)
    input_text = f"{dialogue} []<eos>"  # Assuming empty event tags for simplicity

    # Tokenize the input
    inputs = tokenizer.encode_plus(
        input_text, return_tensors="pt"
    ).to(device)

    # Generate the SOAP note
    outputs = model.generate(
        inputs.input_ids,
        max_length=2048,  # Adjust as needed
        num_beams=4,
        temperature=0.7,
        early_stopping=True,
    )

    # Decode the output
    soap_note = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return soap_note

In [29]:
# Save the fine-tuned model
model.save_pretrained(MODEL_PATH)

# Save the tokenizer
tokenizer.save_pretrained(MODEL_PATH)

print(f"Model and tokenizer saved to {MODEL_PATH}")

Model and tokenizer saved to ..\models


In [30]:
sample_dialogue = train_df.iloc[0,0]
generated_note = generate_soap_note(sample_dialogue, model, tokenizer)
print("Generated SOAP Note:")
print(generated_note)

C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requ

Generated SOAP Note:
Doctor: Hello, how can I help you today?
Patient: My son has been having some issues with speech and development. He's 13 years old now.
Doctor: I see. Can you tell me more about his symptoms? Does he have any issues with muscle tone or hypotonia?
Patient: No, he doesn't have hypotonia. But he has mild to moderate speech and developmental delay, and he's been diagnosed with attention deficit disorder.
Doctor: Thank you for sharing that information. We'll run some tests, including an MRI, to get a better understanding of your son's condition. 
(After the tests)
Doctor: The MRI results are in, and I'm glad to say that there are no structural brain anomalies. However, I did notice some physical characteristics. Does your son have any facial features like retrognathia, mild hypertelorism, or a slightly elongated philtrum and thin upper lip?
Patient: Yes, he has all of those features. His hands are also broad and short. And his feet have mild syndactyly of the second an